## Import libraries and load data

In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [2]:
%ls

EDA.ipynb              air_visit_data.csv     sample_submission.csv
README.md              date_info.csv          store_id_relation.csv
air_reserve.csv        hpg_reserve.csv
air_store_info.csv     hpg_store_info.csv


In [112]:
air_reserve = pd.read_csv('air_reserve.csv')
air_store_info = pd.read_csv('air_store_info.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
date_info = pd.read_csv('date_info.csv')
hpg_reserve = pd.read_csv('hpg_reserve.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')

## Merge dataframes

In [113]:
#Merge store information and reservation

df_air = air_reserve.merge(air_store_info, on='air_store_id', how='left')
df_hpg = hpg_reserve.merge(hpg_store_info, on='hpg_store_id', how='left')

In [114]:
df_air.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_genre_name,air_area_name,latitude,longitude
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819


In [115]:
df_hpg.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,NaN,NaN,NaN,NaN
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,NaN,NaN,NaN,NaN
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,NaN,NaN,NaN,NaN
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,NaN,NaN,NaN,NaN


In [116]:
#Include store id relation

df_air = df_air.merge(store_id_relation, on='air_store_id', how='left')
df_hpg = df_hpg.merge(store_id_relation, on='hpg_store_id', how='left')

In [117]:
df_air_full = df_air.merge(df_hpg, on=['air_store_id', 'visit_datetime','reserve_datetime'], how='outer')
df_hpg_full = df_hpg.merge(df_air, on=['hpg_store_id', 'visit_datetime','reserve_datetime'], how='outer')

Not all air store has a hpg store and vice versa...must combine both dataframes after imputation

## Imputation

In [200]:
#function to impute missing values from air to hpg
def fill_air_hpg(data, feature, is_string=False):
    feature_x = feature + '_x'
    feature_y = feature + '_y'
    
    #if feature type is string like hpg_store_id, check for NaNs with type
    if is_string == True:
        data[feature_x] = data.apply(
            lambda row: row[feature_y] if type(row[feature_x])!=str else row[feature_x], axis=1)
        data[feature_y] = data.apply(
            lambda row: row[feature_x] if type(row[feature_y])!=str else row[feature_y], axis=1)
    #if feature type is float, check for NaNs using math.isnan()
    else:
        data[feature_x] = data.apply(
            lambda row: row[feature_y] if math.isnan(row[feature_x])==True else row[feature_x], axis=1)
        data[feature_y] = data.apply(
            lambda row: row[feature_x] if math.isnan(row[feature_y])==True else row[feature_y], axis=1)

In [214]:
for feature in ['reserve_visitors', 'latitude', 'longitude', 'hpg_store_id']:
    if feature == 'hpg_store_id':
        fill_air_hpg(df_air_test, feature, is_string=True)
    else:
        fill_air_hpg(df_air_test, feature)

In [211]:
df_air_test = df_air_full.copy()
df_air_test = df_air_test.iloc[:100,:]

In [185]:
df_air_test['reserve_visitors_y'] = df_air_test.apply(
    lambda x: x['reserve_visitors_x'] if math.isnan(x['reserve_visitors_y'])==True else x['reserve_visitors_y'], axis=1)
df_air_test['reserve_visitors_x'] = df_air_test.apply(
    lambda x: x['reserve_visitors_y'] if math.isnan(x['reserve_visitors_x'])==True else x['reserve_visitors_x'], axis=1)

In [187]:
df_air_test['latitude_y'] = df_air_test.apply(
    lambda x: x['latitude_x'] if math.isnan(x['latitude_y'])==True else x['latitude_x'], axis=1)
df_air_test['latitude_x'] = df_air_test.apply(
    lambda x: x['latitude_y'] if math.isnan(x['latitude_x'])==True else x['latitude_y'], axis=1)

In [190]:
df_air_test['longitude_y'] = df_air_test.apply(
    lambda x: x['longitude_x'] if math.isnan(x['longitude_y'])==True else x['longitude_x'], axis=1)
df_air_test['longitude_x'] = df_air_test.apply(
    lambda x: x['longitude_y'] if math.isnan(x['longitude_x'])==True else x['longitude_y'], axis=1)

In [188]:
df_air_test['hpg_store_id_y'] = df_air_test.apply(
    lambda x: x['hpg_store_id_x'] if type(x['hpg_store_id_y'])!=str else x['hpg_store_id_y'], axis=1)

df_air_test['hpg_store_id_x'] = df_air_test.apply(
    lambda x: x['hpg_store_id_y'] if type(x['hpg_store_id_x'])!=str else x['hpg_store_id_x'], axis=1)

In [191]:
df_air_test

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id_x,hpg_store_id_y,reserve_visitors_y,hpg_genre_name,hpg_area_name,latitude_y,longitude_y
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,1.0,NaN,NaN,35.658068,139.751599
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,hpg_bcecaa70222d94e5,3.0,NaN,NaN,34.676231,135.486059
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,hpg_bcecaa70222d94e5,6.0,NaN,NaN,34.676231,135.486059
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,2.0,NaN,NaN,35.658068,139.751599
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,hpg_878cc70b1abc76f7,5.0,NaN,NaN,43.770635,142.364819
5,air_db80363d35f10926,2016-01-02 01:00:00,2016-01-01 16:00:00,2.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,hpg_878cc70b1abc76f7,2.0,NaN,NaN,43.770635,142.364819
6,air_db80363d35f10926,2016-01-02 01:00:00,2016-01-01 15:00:00,4.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,hpg_878cc70b1abc76f7,4.0,NaN,NaN,43.770635,142.364819
7,air_3bb99a1fe0583897,2016-01-02 16:00:00,2016-01-02 14:00:00,2.0,Izakaya,Miyagi-ken Sendai-shi Kamisugi,38.269076,140.870403,NaN,NaN,2.0,NaN,NaN,38.269076,140.870403
8,air_3bb99a1fe0583897,2016-01-02 16:00:00,2016-01-01 20:00:00,2.0,Izakaya,Miyagi-ken Sendai-shi Kamisugi,38.269076,140.870403,NaN,NaN,2.0,NaN,NaN,38.269076,140.870403
9,air_2b8b29ddfd35018e,2016-01-02 17:00:00,2016-01-02 17:00:00,2.0,Okonomiyaki/Monja/Teppanyaki,Hiroshima-ken Hiroshima-shi Kokutaijimachi,34.386245,132.455018,hpg_2d4200d0ebdb4946,hpg_2d4200d0ebdb4946,2.0,NaN,NaN,34.386245,132.455018


In [162]:
df_air_test

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id_x,hpg_store_id_y,reserve_visitors_y,hpg_genre_name,hpg_area_name,latitude_y,longitude_y
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,air_877f79706adbfb06,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,1,NaN,NaN,NaN,NaN
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,air_db4b38ebe7a7ceff,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,NaN,3,NaN,NaN,NaN,NaN
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,air_db4b38ebe7a7ceff,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,NaN,6,NaN,NaN,NaN,NaN
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,air_877f79706adbfb06,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,2,NaN,NaN,NaN,NaN
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,air_db80363d35f10926,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,NaN,5,NaN,NaN,NaN,NaN
5,air_db80363d35f10926,2016-01-02 01:00:00,2016-01-01 16:00:00,air_db80363d35f10926,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,NaN,2,NaN,NaN,NaN,NaN
6,air_db80363d35f10926,2016-01-02 01:00:00,2016-01-01 15:00:00,air_db80363d35f10926,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,NaN,4,NaN,NaN,NaN,NaN
7,air_3bb99a1fe0583897,2016-01-02 16:00:00,2016-01-02 14:00:00,air_3bb99a1fe0583897,Izakaya,Miyagi-ken Sendai-shi Kamisugi,38.269076,140.870403,NaN,NaN,2,NaN,NaN,NaN,NaN
8,air_3bb99a1fe0583897,2016-01-02 16:00:00,2016-01-01 20:00:00,air_3bb99a1fe0583897,Izakaya,Miyagi-ken Sendai-shi Kamisugi,38.269076,140.870403,NaN,NaN,2,NaN,NaN,NaN,NaN
9,air_2b8b29ddfd35018e,2016-01-02 17:00:00,2016-01-02 17:00:00,air_2b8b29ddfd35018e,Okonomiyaki/Monja/Teppanyaki,Hiroshima-ken Hiroshima-shi Kokutaijimachi,34.386245,132.455018,hpg_2d4200d0ebdb4946,NaN,2,NaN,NaN,NaN,NaN


In [143]:
#function to fill reserve visitors
def fill_reserve(row, fill='reserve_visitors'):
    if math.isnan(row['reserve_visitors_y']):
        return row['reserve_visitors_x']
    elif math.isnan(row['reserve_visitors_x']):
        return row['reserve_visitors_y']

def fill_latitude
    


In [88]:
df_air_full.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id_x,hpg_store_id_y,reserve_visitors_y,hpg_genre_name,hpg_area_name,latitude_y,longitude_y
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,NaN,NaN,NaN,NaN,NaN,NaN
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,NaN,NaN,NaN,NaN,NaN,NaN
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df_air_full.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id_x,hpg_store_id_y,reserve_visitors_y,hpg_genre_name,hpg_area_name,latitude_y,longitude_y
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,NaN,NaN,NaN,NaN,NaN,NaN
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6.0,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5,NaN,NaN,NaN,NaN,NaN,NaN
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5.0,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df_air_full.hpg_store_id_x.nunique()

131

In [68]:
df_hpg_full.air_store_id_x.nunique()

150

In [44]:
df_hpg.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,hpg_genre_name,hpg_area_name,latitude,longitude,air_store_id
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,NaN,NaN,NaN,NaN,NaN
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698,NaN
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,NaN,NaN,NaN,NaN,NaN
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,NaN,NaN,NaN,NaN,NaN
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,NaN,NaN,NaN,NaN,NaN


In [43]:
df_air.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_genre_name,air_area_name,latitude,longitude,hpg_store_id
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,Dining bar,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059,hpg_bcecaa70222d94e5
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,hpg_878cc70b1abc76f7
